# Survival Analysis Lab

Complete the following exercises to solidify your knowledge of survival analysis.

In [3]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from lifelines import KaplanMeierFitter

cf.go_offline()

In [22]:
pd.options.display.max_columns = None
data = pd.read_csv('../data/attrition.csv')
data.head()



Age  Attrition     BusinessTravel  DailyRate              Department  \
0   41          1      Travel_Rarely       1102                   Sales   
1   49          0  Travel_Frequently        279  Research & Development   
2   37          1      Travel_Rarely       1373  Research & Development   
3   33          0  Travel_Frequently       1392  Research & Development   
4   27          0      Travel_Rarely        591  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  \
0                 1          2  Life Sciences              1               1   
1                 8          1  Life Sciences              1               2   
2                 2          2          Other              1               4   
3                 3          4  Life Sciences              1               5   
4                 2          1        Medical              1               7   

   EnvironmentSatisfaction  Gender  HourlyRate  JobInvolvement  JobLevel  \
0                        2  Female          94               3         2   
1                        3    Male          61               2         2   
2                        4    Male          92               2         1   
3                        4  Female          56               3         1   
4                        1    Male          40               3         1   

                 JobRole  JobSatisfaction MaritalStatus  MonthlyIncome  \
0        Sales Executive                4        Single           5993   
1     Research Scientist                2       Married           5130   
2  Laboratory Technician                3        Single           2090   
3     Research Scientist                3       Married           2909   
4  Laboratory Technician                2       Married           3468   

   MonthlyRate  NumCompaniesWorked Over18 OverTime  PercentSalaryHike  \
0        19479                   8      Y      Yes                 11   
1        24907                   1      Y       No                 23   
2         2396                   6      Y      Yes                 15   
3        23159                   1      Y      Yes                 11   
4        16632                   9      Y       No                 12   

   PerformanceRating  RelationshipSatisfaction  StandardHours  \
0                  3                         1             80   
1                  4                         4             80   
2                  3                         2             80   
3                  3                         3             80   
4                  3                         4             80   

   StockOptionLevel  TotalWorkingYears  TrainingTimesLastYear  \
0                 0                  8                      0   
1                 1                 10                      3   
2                 0                  7                      3   
3                 0                  8                      3   
4                 1                  6                      3   

   WorkLifeBalance  YearsAtCompany  YearsInCurrentRole  \
0                1               6                   4   
1                3              10                   7   
2                3               0                   0   
3                3               8                   7   
4                3               2                   2   

   YearsSinceLastPromotion  YearsWithCurrManager  
0                        0                     5  
1                        1                     7  
2                        0                     0  
3                        3                     0  
4                        2                     2

## 1. Generate and plot a survival function that shows how employee retention rates vary by gender and employee age.

*Tip: If your lines have gaps in them, you can fill them in by using the `fillna(method=ffill)` and the `fillna(method=bfill)` methods and then taking the average. We have provided you with a revised survival function below that you can use for the exercises in this lab*

In [10]:
def survival(data, group_field, time_field, event_field):
    kmf = KaplanMeierFitter()
    results = []

    for i in data[group_field].unique():
        group = data[data[group_field]==i]
        T = group[time_field]
        E = group[event_field]
        kmf.fit(T, E, label=str(i))
        results.append(kmf.survival_function_)

    survival = pd.concat(results, axis=1)
    front_fill = survival.fillna(method='ffill')
    back_fill = survival.fillna(method='bfill')
    smoothed = (front_fill + back_fill) / 2
    return smoothed

In [36]:
rates = survival(data, 'Gender', 'Age','Attrition')

rates.iplot(kind='line', xTitle='Age', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Gender')

## 2. Compare the plot above with one that plots employee retention rates by gender over the number of years the employee has been working for the company.

In [32]:
rates = survival(data, 'Gender', 'TotalWorkingYears','Attrition')

rates.iplot(kind='line', xTitle='TotalWorkingYears', yTitle='Retention Rate',
            title='Retention Rates by Gender and Total Working Years')

## 3. Let's look at retention rate by gender from a third perspective - the number of years since the employee's last promotion. Generate and plot a survival curve showing this.

In [33]:
rates = survival(data, 'Gender', 'YearsSinceLastPromotion','Attrition')

rates.iplot(kind='line', xTitle='YearsSinceLastPromotion', yTitle='Retention Rate',
            title='Retention Rates by Gender and Years since last promotion')

## 4. Let's switch to looking at retention rates from another demographic perspective: marital status. Generate and plot survival curves for the different marital statuses by number of years at the company.

In [34]:
rates = survival(data, 'MaritalStatus', 'TotalWorkingYears','Attrition')

rates.iplot(kind='line', xTitle='Total Working Years', yTitle='Retention Rate',
            title='Retention Rates by Marital Status and Total Working Years')

## 5. Let's also look at the marital status curves by employee age. Generate and plot the survival curves showing retention rates by marital status and age.

In [77]:
rates = survival(data, 'MaritalStatus', 'Age','Attrition')

rates.iplot(kind='line', xTitle='Age', yTitle='Retention Rate',
            title='Retention Rates by Marital Status by employee age')

## 6. Now that we have looked at the retention rates by gender and marital status individually, let's look at them together. 

Create a new field in the data set that concatenates marital status and gender, and then generate and plot a survival curve that shows the retention by this new field over the age of the employee.

In [78]:
data['status_gender']=data['Gender']+' '+data['MaritalStatus']

rates = survival(data, 'status_gender', 'Age','Attrition')

rates.iplot(kind='line', xTitle='Age', yTitle='Retention Rate',
            title='Retention Rates by Marital Status and gender vs Age of employee')


## 6. Let's find out how job satisfaction affects retention rates. Generate and plot survival curves for each level of job satisfaction by number of years at the company.

In [80]:
rates = survival(data, 'JobSatisfaction', 'YearsAtCompany','Attrition')

rates.iplot(kind='line', xTitle='YearsAtCompany', yTitle='Retention Rate',
            title='Retention Rates by Job Satisfaction and number of years at the company')


## 7. Let's investigate whether the department the employee works in has an impact on how long they stay with the company. Generate and plot survival curves showing retention by department and years the employee has worked at the company.

In [81]:
rates = survival(data, 'Department', 'YearsAtCompany','Attrition')

rates.iplot(kind='line', xTitle='YearsAtCompany', yTitle='Retention Rate',
            title='Retention Rates by Department and years has worked at the company')

## 8. From the previous example, it looks like the sales department has the highest attrition. Let's drill down on this and look at what the survival curves for specific job roles within that department look like.

Filter the data set for just the sales department and then generate and plot survival curves by job role and the number of years at the company.

In [49]:
data_filter=data.loc[data.Department=='Sales']
data_filter.head()

rates = survival(data.loc[data.Department=='Sales'], 'JobRole', 'YearsAtCompany','Attrition')

rates.iplot(kind='line', xTitle='YearsAtCompany', yTitle='Retention Rate',
            title='Retention Rates by Job Roles of Sales and years at compan')


## 9. Let examine how compensation affects attrition.

- Use the `pd.qcut` method to bin the HourlyRate field into 5 different pay grade categories (Very Low, Low, Moderate, High, and Very High).
- Generate and plot survival curves showing employee retention by pay grade and age.

In [54]:
data['grade']=pd.qcut(data.HourlyRate,5,labels=['Very Low','Low','Moderate','High','Very High'])

survival(data, 'grade', 'Age','Attrition').iplot(kind='line', xTitle='Age', yTitle='Retention Rate',
            title='Retention Rates by Marital Status and Total Working Years')


## 10. Finally, let's take a look at how the demands of the job impact employee attrition.

- Create a new field whose values are 'Overtime' or 'Regular Hours' depending on whether there is a Yes or a No in the OverTime field.
- Create a new field that concatenates that field with the BusinessTravel field.
- Generate and plot survival curves showing employee retention based on these conditions and employee age.

In [76]:
data['demands']= data['OverTime'].apply(lambda y: 'Overtime' if y=='Yes' else 'Regular Hours')
data['demands_BusinessTravel'] = data['demands']+' '+ data['BusinessTravel']
survival(data, 'demands_BusinessTravel', 'Age','Attrition').iplot(kind='line', xTitle='Age', yTitle='Retention Rate',
            title='Retention Rates by Time at work and Bussines Travel vs Age')